僅地圖


In [2]:
import os
import gpxpy
import json
from typing import List, Tuple

GPX_FOLDER = "hikingbook_route"
OUTPUT_HTML = "show_route.html"


# 讀取資料夾內所有 GPX
def load_gpx_routes(folder: str) -> List[Tuple[str, List[Tuple[float, float]]]]:
    routes = []
    for filename in os.listdir(folder):
        if filename.endswith(".gpx"):
            filepath = os.path.join(folder, filename)
            with open(filepath, "r", encoding="utf-8") as gpx_file:
                gpx = gpxpy.parse(gpx_file)
                points = []
                for track in gpx.tracks:
                    for segment in track.segments:
                        for point in segment.points:
                            points.append((point.latitude, point.longitude))
                if points:
                    routes.append((filename, points))
    return routes


# 產出 HTML
def generate_html(routes: List[Tuple[str, List[Tuple[float, float]]]]):
    route_json = json.dumps(
        [{"name": name, "points": coords} for name, coords in routes]
    )

    html_template = f"""
<!DOCTYPE html>
<html lang="zh-Hant">
<head>
    <meta charset="UTF-8">
    <title>GPX 路線檢視器</title>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css" />
    <style>
        #map {{ height: 90vh; }}
        #checkboxes {{
            height: 10vh;
            overflow-x: auto;
            white-space: nowrap;
            background: #f9f9f9;
            padding: 5px;
        }}
        .route-option {{
            display: inline-block;
            margin-right: 10px;
        }}
    </style>
</head>
<body>
    <div id="map"></div>
    <div id="checkboxes"></div>

    <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
    <script>
        const map = L.map('map').setView([24.0, 121.0], 8);
        L.tileLayer('https://{{s}}.tile.openstreetmap.org/{{z}}/{{x}}/{{y}}.png', {{
            maxZoom: 18
        }}).addTo(map);

        const routes = {route_json};
        const routeLayers = {{}};

        routes.forEach(route => {{
            const latlngs = route.points.map(p => [p[0], p[1]]);
            const polyline = L.polyline(latlngs, {{ color: 'blue' }}).addTo(map);
            routeLayers[route.name] = polyline;

            // 自動調整地圖視野
            map.fitBounds(polyline.getBounds());

            // 建立 checkbox
            const cbContainer = document.getElementById('checkboxes');
            const wrapper = document.createElement('div');
            wrapper.className = 'route-option';

            const checkbox = document.createElement('input');
            checkbox.type = 'checkbox';
            checkbox.checked = true;
            checkbox.id = route.name;
            checkbox.onchange = () => {{
                if (checkbox.checked) {{
                    routeLayers[route.name].addTo(map);
                }} else {{
                    map.removeLayer(routeLayers[route.name]);
                }}
            }};

            const label = document.createElement('label');
            label.htmlFor = route.name;
            label.innerText = route.name;

            wrapper.appendChild(checkbox);
            wrapper.appendChild(label);
            cbContainer.appendChild(wrapper);
        }});
    </script>
</body>
</html>
"""
    with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
        f.write(html_template)
    print(f"✅ 已產生 {OUTPUT_HTML}")


if __name__ == "__main__":
    gpx_routes = load_gpx_routes(GPX_FOLDER)
    if not gpx_routes:
        print("⚠️ 找不到 GPX 路線")
    else:
        generate_html(gpx_routes)

✅ 已產生 show_route.html
